# Web Scraping for Indeed.com & Predicting Salaries

In [122]:
print range(0,60,10)

[0, 10, 20, 30, 40, 50]


In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [123]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=0"

In [124]:
import requests
import bs4
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
import numpy as np

In [125]:
## YOUR CODE HERE

page = urllib2.urlopen(URL)

#parse the page
soup = BeautifulSoup(requests.get(URL).text)

#extracting all results
result = soup.find("div", {"class": "row result"},'lxml')

print result

<div class="row result" data-jk="2ccb5eca0c079ac2" id="pj_2ccb5eca0c079ac2">
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0AVIi8UxprrPGU7QPohOxOOpynq0pcPnEidcD-eE3H2SkUy41FI-O8USBYuwNfR3iWSpf_jEVuUvE5YjIVl-dP1Lgiaoqm2gYK8C18Ed5lsS9xt1V6feICDW65ZRK1lcyqcpm2OQo4Rm70XcVlL8SR33HLSUK1wJ_ORCB-oIuZx-be9QX5q_4XQ8Zv_lJ7Irq5F2_7dmAXNKRb9DTJxBSLZ9whAfqYN2uP2aeRluWyvd-9lP07PIFSDlIihjyeOoLnrCiAeXBW4awA7DN3w9885ZULRWBfjPX_EJy0RkiNJ7CR2o83mAhvnblB4cTAnQEIioveXvHt2C7chNKbP3IpLkk6bxX7zWR5vcjE2L7fvco2v04lMbEaFwYBnPiqw_7ILmdIXY7fwi-8b8QjNGMI00cBc6niKnq9hoJqoMiSGqJ89IkUGCfvLCA6K6YfkcdowQh6VbluuySUAmIwd73Gnhhwozumk6yucXkAvVUmROgjsfECS4eOVdipNVJp2H3Krv0TZaksUZYZ0IZ2ZhKo1gb-Zv64HBg7AOYZ8pwmul5vjd2hOqi4qqAEhU6HwNOiDtBN_2nIqEA_c7zo-pWM2Qc-LqRj9AW-Vt7KyjChek6cybIjPkuly0h0LjfZstZb5iXeP8nodDCq4sBrpx3tDLopN87scpxY=&amp;p=1&amp;sk=&amp;fvj=0" id="sja1" onclick="setRefineByCookie(['salest']); sjoc('sja1',0); convCtr('SJ', pingUrlsForGA)" onmousedown="sjomd('sja1

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

## YOUR CODE HERE
def extract_location_from_result(result):
    for loc in soup.findAll('span', {'class':'location'}):
        return loc.get_text(strip=True)

def extract_company_from_result(result):
    for loc in soup.findAll('span', {'class':'company'}):
        return loc.get_text(strip=True)

salary = []

def extract_salary_from_result(result):
    for loc in soup.findAll('span', {'class': 'location'}):
        Sibling = soup.findNextSiblings('div')
        if Sibling != None:
            salary.append(loc.text.strip())
        else:
            salary.append(np.nan)

def extract_title_from_result(result):
    for loc in soup.findAll('a', {'data-tn-element':'jobTitle'}):
        return loc['title']
    
#print extract_title_from_result(result)


In [126]:
#def extract_text(el):
#    if el:
#        return el.text.strip()
#    else:
#        return ''
        
def extract_company_from_result(result):
    return result.find('span', {'class' : 'company'}).text.strip()

def extract_location_from_result(result):
    return  result.find('span', {'class' : 'location'}).text.strip()

#def extract_summary_from_result(result):
#    return  extract_text(result.find('span', {'class' : 'summary'}))

def extract_title_from_result(result):
    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

def extract_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()
        else:
            return np.nan
        
print extract_salary_from_result(result)

nan


In [127]:
lst1 = [loc.get_text(strip=True) for loc in soup.findAll('span', {'class':'company'})]
print (lst1)

lst2 = [loc.get_text(strip=True) for loc in soup.findAll('span', {'class':'location'})]
print (lst2)

lst3 = [loc.get_text(strip=True) for loc in soup.findAll('a', {'data-tn-element':'jobTitle'})]#dual conditions?
print (lst3)

#lst3 = [loc.findNextSibling('div') for loc in soup.findAll('div', {'class':'sjcl'})]
#print (lst3)

#http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10


[u'Uber', u'Lancer Insurance Company', u'Gallup', u'Memorial Sloan Kettering', u'MakerBot', u'eTemp', u'Memorial Sloan Kettering Cancer Center', u'Comcast', u'DataKind', u'Samsung Accelerator', u'Illinois Technology Association', u'Bloomberg', u'Amazon Web Services, Inc.', u'Jackson Lewis P.C.', u'AbilTo, Inc']
[u'New York, NY', u'Long Beach, NY', u'New York, NY', u'New York, NY', u'New York, NY', u'New York, NY', u'New York, NY 10065(Upper East Side area)', u'New York, NY', u'New York, NY', u'New York, NY', u'New York, NY', u'New York, NY', u'New York, NY', u'Melville, NY', u'New York, NY']
[u'DataScientist- Monitoring & Anomaly Detection Platform', u'DataScientist& Analytics Developer', u'DataScientist\u2013 Predictive Analytics', u'Bioinformatics Engineer', u'DataScientist', u'Mid - Senior LevelDataScientist', u'BioinformaticsDataScientist- Cancer Genomics, NYC', u'DataScientist', u'DataScientist', u'Machine Learning Engineer', u'GrubHub -DataScientist', u'Machine Learning Engineer'

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [128]:
YOUR_CITY = 'Atlanta'

In [129]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%24{}%2C000&l={}&start={}"
max_results_per_city = 500

rows = []


for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Los+Angeles',YOUR_CITY]):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        #parse the page
        for salary in set([0,100]):
            url = url_template.format(salary,city,start)
            soup = BeautifulSoup(requests.get(url).text)
            #extracting all results
            results = soup.findAll("div", {"class": "row result"})
       
            if salary == 0:
                for result in results:
                    if result:
                        row = {}
                        row['title'] =  extract_title_from_result(result)
                        row['company'] = extract_company_from_result(result)
                        row['salary'] = 0
                        row['city'] = city
                        rows.append(row)
                   
            else:
                for result in results:
                    if result:
                        row = {}
                        row['title'] =  extract_title_from_result(result)
                        row['company'] = extract_company_from_result(result)
                        row['salary'] = 1
                        row['city'] = city
                        rows.append(row)
                    

In [130]:
rows

[{'city': 'Chicago',
  'company': u'Gallup',
  'salary': 0,
  'title': u'Data Scientist \u2013 Predictive Analytics'},
 {'city': 'Chicago',
  'company': u'Allstate Insurance',
  'salary': 0,
  'title': u'Senior Telematics Data Scientist'},
 {'city': 'Chicago',
  'company': u'Next Step Systems',
  'salary': 0,
  'title': u'Junior Quantitative Research Analyst'},
 {'city': 'Chicago',
  'company': u'Gallup',
  'salary': 1,
  'title': u'Data Scientist \u2013 Predictive Analytics'},
 {'city': 'Chicago',
  'company': u'Next Step Systems',
  'salary': 1,
  'title': u'Junior Quantitative Research Analyst'},
 {'city': 'Chicago',
  'company': u'Allstate Insurance',
  'salary': 1,
  'title': u'Senior Telematics Data Scientist'},
 {'city': 'Chicago',
  'company': u'Next Step Systems',
  'salary': 0,
  'title': u'Junior Quantitative Research Analyst'},
 {'city': 'Chicago',
  'company': u'Allstate Insurance',
  'salary': 0,
  'title': u'Data Science Manager'},
 {'city': 'Chicago',
  'company': u'All

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [131]:
## YOUR CODE HERE

data = pd.DataFrame.from_records(rows)


#data.describe()
#data.sort_values(by='salary')
data = data.drop_duplicates(['city','company','title'],keep='last')
data

,city,company,salary,title
221,Chicago,Parallel Partners,1,Quantitative Research Analyst - MS061333
436,Chicago,Catalina Marketing,0,"Senior Analyst, Data Science"
440,Chicago,United Airlines,0,Senior Analyst - Statistics and Operations Res...
447,Chicago,Parallel Partners,1,Junior Quantitative Research Analyst - MS062161
448,Chicago,Ginas Tech Jobs,1,Quantitative Research Analyst
451,Chicago,Thermo Fisher Scientific,0,Applications Scientist
452,Chicago,Sysmex,0,"Scientist I, Technical Support - Sysmex Inostics"
453,Chicago,Gallup,0,Data Scientist – Predictive Analytics
454,Chicago,Allstate Insurance,0,Machine Learning Engineer - Connected Car
456,Chicago,"ACGT, Inc.",0,Bioinformatics Analyst


Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [132]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [133]:
## YOUR CODE HERE

### Save your results as a CSV

In [134]:
## YOUR CODE HERE
data.to_csv('../assets/cleanIndeed.csv',encoding='utf-8')

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [135]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [136]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [137]:
## YOUR CODE HERE

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [138]:
dummy_location = pd.get_dummies(data['city'], prefix='city')
cols_to_keep = ['company', 'salary', 'title']
handCalc = data[cols_to_keep].join(dummy_location.ix[:, :])
#print handCalc.head()
#print pd.crosstab(handCalc['salary'], handCalc['city_New+York'], rownames=['salary'])

In [139]:
## YOUR CODE HERE
import statsmodels.api as sm
cols_to_keep = ['city', 'salary', 'title']
df = data[cols_to_keep].join(dummy_location.ix[:, 'city_Austin':])
#print df.head()
df['intercept'] = 1.0
#print df.ix[:, df.columns != 'salary']

logit = sm.Logit(df['salary'], df.ix[:,3:])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.645408
         Iterations 5


In [140]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                 salary   No. Observations:                   99
Model:                          Logit   Df Residuals:                       93
Method:                           MLE   Df Model:                            5
Date:                Wed, 29 Jun 2016   Pseudo R-squ.:                 0.05313
Time:                        23:15:14   Log-Likelihood:                -63.895
converged:                       True   LL-Null:                       -67.481
                                        LLR p-value:                    0.2082
                         coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
city_Austin            1.3218      0.753      1.756      0.079        -0.154     2.797
city_Chicago           0.2231      0.689      0.324      0.746        -1.128     1.574
city_Los+Angeles    

In [141]:
print np.exp(result.params)
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

city_Austin           3.750000
city_Chicago          1.250000
city_Los+Angeles      1.750000
city_New+York         0.833333
city_San+Francisco    0.625000
intercept             1.000000
dtype: float64
                        2.5%      97.5%        OR
city_Austin         0.857571  16.398062  3.750000
city_Chicago        0.323787   4.825699  1.250000
city_Los+Angeles    0.472395   6.482925  1.750000
city_New+York       0.178808   3.883751  0.833333
city_San+Francisco  0.141392   2.762712  0.625000
intercept           0.375318   2.664408  1.000000


In order to get a 6-figure salary, it's best to go to these cities: Austin, Chicago, LA, Atlanta, New York, San Francisco

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [142]:
## YOUR CODE HERE

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [143]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [144]:
## YOUR CODE HERE

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [145]:
## YOUR CODE HERE

In [146]:
## YOUR CODE HERE

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [147]:
## YOUR CODE HERE

In [148]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [149]:
## YOUR CODE HERE